In [1]:
import pandas as pd
import numpy as np
import glob
import re

In [2]:
csvs = glob.glob('../data/KmerMixing/bac_meta/csv/*.csv')
dat = list()
for e in csvs:
    df = pd.read_csv(e, delimiter=';')
    df = df.loc[df.Population.notnull(), :]
    df.Population = df.Population.astype(int)
    df['gen'] = int(re.findall('Sequences([0-9]+)-', e)[0])
    dat.append(df)
allseqs = pd.concat(dat)
allseqs.reset_index(drop=True, inplace=True)

In [3]:
genes = [e for e in allseqs.columns if 'Gene' in e]
allseqs['fa'] = allseqs.apply(lambda r: ''.join(e.replace('(', '').replace(')', '').replace('-', '').upper() for e in r[genes]),  axis=1)
allseqs['mutseq'] = allseqs.apply(lambda r: ''.join(e for e in r[genes]),  axis=1)
allseqs['fa'].apply(lambda x: len(x))

0      4000000
1      4000000
2      4000000
3      4000000
4      4000000
        ...   
195    4000000
196    4000000
197    4000000
198    4000000
199    4000000
Name: fa, Length: 200, dtype: int64

In [4]:
# write fasta
for i in allseqs.index:
    gen = allseqs.loc[i, 'gen']
    pop = allseqs.loc[i, 'Population']
    mem = i % 100
    fa = allseqs.loc[i, 'fa']
    ms = allseqs.loc[i, 'mutseq']
    with open(f'../data/KmerMixing/bac_meta/fasta/{gen}-{pop}-{mem}.fa', 'w') as f:
        f.write(f'>{gen}-{pop}-{mem}\n{fa}\n')
    with open(f'../data/KmerMixing/bac_meta/fasta/{gen}-{pop}-{mem}.ms.txt', 'w') as f:
        f.write(f'>{gen}-{pop}-{mem}\n{ms}\n')

# write csv
allseqs.to_csv('../data/KmerMixing/bac_meta/allseqs.csv')